In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Data
df = pd.read_csv('skygeni_sales_data.csv')

**Feature Engineering**

In [ ]:
# We need to turn dates into numbers the model can understand.

# Convert dates
df['created_date'] = pd.to_datetime(df['created_date'])
df['closed_date'] = pd.to_datetime(df['closed_date'])

# Feature 1: Deal Age (How long has it been open?)
# For closed deals: closed_date - created_date
# For open deals: Today - created_date
today = pd.to_datetime('today')
df['deal_age_days'] = (df['closed_date'].fillna(today) - df['created_date']).dt.days

# Feature 2: Stage Encoding (Ordinal is better than One-Hot here)
# Higher stage = Closer to winning (usually good, unless age is high)
stage_map = {'Qualified': 1, 'Demo': 2, 'Proposal': 3, 'Negotiation': 4, 'Closed': 5}
df['stage_score'] = df['deal_stage'].map(stage_map).fillna(0)

# Feature 3: Region Encoding (One-Hot)
# We use get_dummies to turn 'Region' into 'Region_West', 'Region_East', etc.
df_encoded = pd.get_dummies(df, columns=['region', 'product_type'], drop_first=True)

**Splitting the data (Training Vs Testing)**

For Training, I have used the data that falls under the year 2023 (~80% of dataset)

And for Testing, remaining data that falls under the year 2024 (~20% of dataset)

In [ ]:
# 1. TRAINING SET: Using all data Before 2024
train_df = df_encoded[df_encoded['created_date'] < '2024-01-01'].copy()

# Target: 1 = Lost (Risk), 0 = Won (Safe)
train_df['target_risk'] = train_df['outcome'].apply(lambda x: 1 if x == 'Lost' else 0)

# 2. TESTING SET: Use data From 2024

# We treat these as if we don't know the outcome yet
open_deals = df_encoded[df_encoded['created_date'] >= '2024-01-01'].copy()

# Select Features (Exclude dates and IDs)
features = ['deal_amount', 'deal_age_days', 'stage_score'] + \
           [col for col in df_encoded.columns if 'region_' in col or 'product_type_' in col]

X_train = train_df[features]
y_train = train_df['target_risk']

**Training the Model on year 2023 data**

In [ ]:
# Initialize Random Forest
print(f"Training on {len(train_df)} historical deals.")
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

Training on 4039 historical deals.


RandomForestClassifier(max_depth=10, random_state=42)

**Testing the Model on year 2024 data**

In [ ]:
# --- SCORING THE "OPEN" PIPELINE ---
print(f"Scoring {len(open_deals)} current Q1 2024 deals.")

# Get Probability of Loss (Risk Score)
risk_scores = rf_model.predict_proba(open_deals[features])[:, 1]
open_deals['risk_score'] = (risk_scores * 100).round(1)

# Categorize Risk
def categorize_risk(score):
    if score > 75: return 'Critical'
    if score > 50: return 'High'
    return 'Normal'

open_deals['risk_category'] = open_deals['risk_score'].apply(categorize_risk)

Scoring 961 current Q1 deals...


**Generating Final Actionable Output**

In [ ]:
# Sort by Risk (Highest first) AND Deal Size (Highest first)
hit_list = open_deals.sort_values(by=['risk_score', 'deal_amount'], ascending=[False, False])

print("\n🚨 CRO ALERT: TOP 5 AT-RISK DEALS")
cols_to_show = ['deal_id', 'deal_amount', 'deal_stage', 'deal_age_days', 'risk_score', 'risk_category']

# Print the first few columns + risk
print(hit_list[['deal_id', 'deal_amount', 'deal_stage', 'region_North America', 'deal_age_days', 'risk_score', 'risk_category']].head(5))

# FEATURE IMPORTANCE
importances = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\n🧠 MODEL INSIGHT: WHAT DRIVES RISK?")
print(importances.head(3))


🚨 CRO ALERT: TOP 5 AT-RISK DEALS
     deal_id  deal_amount deal_stage  region_North America  deal_age_days  \
4871  D04872        13128  Qualified                  True             96   
1608  D01609        19539  Qualified                  True             99   
4198  D04199         7634  Qualified                  True              9   
2920  D02921         8361  Qualified                  True             62   
451   D00452        11866  Qualified                  True             92   

      risk_score risk_category  
4871        83.3      Critical  
1608        79.4      Critical  
4198        78.6      Critical  
2920        78.5      Critical  
451         78.1      Critical  

🧠 MODEL INSIGHT: WHAT DRIVES RISK?
         Feature  Importance
0    deal_amount    0.426086
1  deal_age_days    0.377217
2    stage_score    0.091801
